In [1]:
import pandas as pd
routes = pd.read_csv('routes.txt', sep=",")
trips = pd.read_csv('trips.txt', sep=",")
stop_times = pd.read_csv('stop_times.txt', sep=",")
calendar = pd.read_csv('calendar.txt', sep=",")

stop_times['start_id'] = stop_times['stop_id'].shift()
stop_times['start_time_from_last_station'] = stop_times['departure_time'].shift()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
final = pd.merge(stop_times, trips, on='trip_id')

final = pd.merge(final, routes, on='route_id')

In [3]:
final2 = pd.merge(final, calendar, on='service_id')

In [4]:

final2['start_id'] = final['stop_id'].shift()
final2['start_time_from_last_station'] = final['departure_time'].shift()

In [5]:

import numpy as np
final2.loc[final2['stop_sequence'] == 0, 'start_id'] = np.NAN
final2.loc[final2['stop_sequence'] == 0, 'start_time_from_last_station'] = np.NAN

In [6]:
final2 = final2[final2['start_id'].notna()]

In [7]:
#final2['start_date'] = final2['start_date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
#final2['end_date'] = final2['end_date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d')) 

In [8]:
final2 = final2.drop(columns=['service_id', 'trip_id', 'trip_headsign', 'trip_short_name', 'direction_id', 'block_id', 'wheelchair_accessible', 'bikes_allowed', 'trip_desc', 'shape_id', 'agency_id', 'route_desc'])

final2 = final2.drop(columns=['stop_time_desc', 'pickup_type', 'drop_off_type'])


In [9]:
final2 = final2.drop(columns=['route_long_name', 'route_url', 'route_color', 'route_text_color'])


In [10]:
final2['new_arrival_time'] = '01 ' + final2['arrival_time']

final2.loc[final2['arrival_time'].str.contains('^24:'), 'new_arrival_time'] = '02 00:' + final2['arrival_time'].str[3:]
final2.loc[final2['arrival_time'].str.contains('^25:'), 'new_arrival_time'] = '02 01:' + final2['arrival_time'].str[3:]
final2.loc[final2['arrival_time'].str.contains('^26:'), 'new_arrival_time'] = '02 02:' + final2['arrival_time'].str[3:]
final2.loc[final2['arrival_time'].str.contains('^27:'), 'new_arrival_time'] = '02 03:' + final2['arrival_time'].str[3:]

print('half')


half


In [11]:

final2['new_arrival_time'] =  pd.to_datetime(final2['new_arrival_time'], format='%d %H:%M:%S')


In [12]:
final2['new_departure_time'] = '01 ' + final2['departure_time']

final2.loc[final2['departure_time'].str.contains('^24:'), 'new_departure_time'] = '02 00:' + final2['departure_time'].str[3:]
final2.loc[final2['departure_time'].str.contains('^25:'), 'new_departure_time'] = '02 01:' + final2['departure_time'].str[3:]
final2.loc[final2['departure_time'].str.contains('^26:'), 'new_departure_time'] = '02 02:' + final2['departure_time'].str[3:]
final2.loc[final2['departure_time'].str.contains('^27:'), 'new_departure_time'] = '02 03:' + final2['departure_time'].str[3:]

final2['new_departure_time'] =  pd.to_datetime(final2['new_departure_time'], format='%d %H:%M:%S')




In [13]:
final2['new_start_time_from_last_station'] = '01 ' + final2['start_time_from_last_station']
final2.loc[final2['start_time_from_last_station'].str.contains('^24:'), 'new_start_time_from_last_station'] = '02 00:' + final2['start_time_from_last_station'].str[3:]
final2.loc[final2['start_time_from_last_station'].str.contains('^25:'), 'new_start_time_from_last_station'] = '02 01:' + final2['start_time_from_last_station'].str[3:]
final2.loc[final2['start_time_from_last_station'].str.contains('^26:'), 'new_start_time_from_last_station'] = '02 02:' + final2['start_time_from_last_station'].str[3:]
final2.loc[final2['start_time_from_last_station'].str.contains('^27:'), 'new_start_time_from_last_station'] = '02 03:' + final2['start_time_from_last_station'].str[3:]
final2['new_start_time_from_last_station'] =  pd.to_datetime(final2['new_start_time_from_last_station'], format='%d %H:%M:%S')


In [14]:
del final2['arrival_time']
del final2['departure_time']
del final2['start_time_from_last_station']
del final2['stop_sequence']


In [16]:
final2['new_arrival_time'] = (pd.to_datetime(final2['new_arrival_time']).dt.day * 24 + pd.to_datetime(final2['new_arrival_time']).dt.hour) * 3600 + pd.to_datetime(final2['new_arrival_time']).dt.minute * 60

final2['new_start_time_from_last_station'] = (pd.to_datetime(final2['new_start_time_from_last_station']).dt.day * 24 + pd.to_datetime(final2['new_start_time_from_last_station']).dt.hour) * 3600 + pd.to_datetime(final2['new_start_time_from_last_station']).dt.minute * 60




In [17]:
final2['new_departure_time'] = (pd.to_datetime(final2['new_departure_time']).dt.day * 24 + pd.to_datetime(final2['new_departure_time']).dt.hour) * 3600 + pd.to_datetime(final2['new_departure_time']).dt.minute * 60




In [18]:
stops = final2.to_csv('transits.csv', index=False)